In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import getpass

if getpass.getuser() == 'benjamin.lebrun':
    sys.path.append('/home/mila/b/benjamin.lebrun/genparse')
    os.environ['HF_HOME'] = os.path.join(os.environ['SCRATCH'], 'hf_cache')
    print('HF cache set; path updated')

import nest_asyncio

nest_asyncio.apply()

HF cache set; path updated


In [17]:
prompt = """
    You have access to a political survey data table named "data", which includes the following columns:
    - "age" (integer)
    - "gender" ("male" or "female"),
    - "year" (integer)
    - "state_color" ("blue" or "red")
    - "zipcode" (integer)
    - "vote" ("democrat" or "republican") 
    - "race_ethnicity" ("white", "black", or "latino").

    Q: Write a SQL query that shows individuals' age and gender, for people over 50 years old.
    A: SELECT age, gender FROM data WHERE age>50 </s>
    Q: Write a SQL query that shows individuals' vote and zipcode, ordered from lowest to highest age.
    A: SELECT vote, zipcode, age FROM data ORDER BY age ASC </s>
    Q: Write a SQL query that returns white voters' average age for each state color. 
    A:"""

very_restricted_sql = r"""
    start: WS? "SELECT" WS select_expr WS "FROM" WS from_expr [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
    EOS: "</s>"
    select_expr: STAR | select_list
    bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")"
    bool_expr: var "=" value | var ">" value | var "<" value
    from_expr: "data"
    orderby_expr: var_list WS "ASC" | var_list WS "DESC"
    select_list: select_var ("," WS select_var)*
    var_list: var ("," WS var)*
    select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
    var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
    value: NUMBER | "'red'" | "'blue'" | "'white'" | "'black'" | "'latino'" | "'republican'" | "'democrat'" | "'male'" | "'female'"
    STAR: "*"
    NUMBER: /\d+/
    WS: /[ ]/
"""

In [22]:
from genparse.lm import AsyncGreedilyTokenizedLLM

genparse_llm = AsyncGreedilyTokenizedLLM.from_name(
    'codellama/CodeLlama-7b-Instruct-hf', batch_size=40
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.26s/it]


In [35]:
from genparse.cfglm import EarleyBoolMaskCFGLM
from genparse.util import LarkStuff

guide = EarleyBoolMaskCFGLM(LarkStuff(very_restricted_sql).char_cfg(0.99, ignore='[ ]?'))

In [37]:
from genparse.steer import HFPPLSampler
from genparse.proposal import CharacterProposal

sampler = HFPPLSampler(llm=genparse_llm, guide=guide)
proposal = CharacterProposal(llm=genparse_llm, guide=guide)

In [38]:
particle_approx = sampler.run_inference(
    prompt=prompt,
    proposal=proposal,
    method='smc-standard',
    n_particles=5,
    max_tokens=50,
    verbosity=1,
)

` SELECT` :  SELECT : -0.06184182748828748
` SELECT` :  SELECT : -0.06184182748828748
` SELECT` :  SELECT : -0.06184182748828748
` ` :   : -0.06184182748828748
` SELECT` :  SELECT : -0.06184182748828748
` ` :  SELECT  : -0.12192296165708294
` state` :  SELECT state : -0.12192296165708269
` state` :  SELECT state : -0.12192296165708269
` SELECT` :   SELECT : -2.428910945487174
` state` :  SELECT state : -0.12192296165708269
` state` :  SELECT  state : -1.0110471333211133
`_` :  SELECT state_ : -0.14568685267732434
`_` :  SELECT state_ : -0.14568685267732434
` state` :   SELECT state : -2.508306421396838
`_` :  SELECT state_ : -0.14568685267732434
`_` :  SELECT  state_ : -1.0428825274653986
`color` :  SELECT state_color : -0.1483368355978381
`color` :  SELECT state_color : -0.1483368355978381
`_` :   SELECT state_ : -2.5346895862731693
`color` :  SELECT state_color : -0.1483368355978381
`color` :  SELECT  state_color : -1.046969649109951
`,` :  SELECT state_color, : -0.15111039615886318


In [39]:
particle_approx.posterior

key,value
"SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color ORDER BY state_color, age ASC </s> ▪",1.0


In [40]:
particle_approx.sample()

" SELECT  state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color  ORDER BY state_color,  age ASC  </s> ▪"